In [1]:
import numpy as np
from scipy.sparse import coo_array, kron
from collections import defaultdict
from solver import lanczos_iteration

In [2]:
A = np.load('problem_10_1.npy')
F = A.shape[1]
R = A.shape[0]
lamb = 0

c = np.zeros((R))

# compute helper variables
b = np.ones((F))
aat = A.dot(A.T)

h = defaultdict(int)
J = defaultdict(int)

for (i, occs) in enumerate(aat):
  h[i] = 1/2 * np.sum(occs) - np.sum(A[i]) + 1/2 * lamb * c[i]

for j in range(R):
  for i in range(j):
    J[(i, j)] = 1/2 * aat[i, j]

pauli_0 = coo_array(([1, 1], ((0, 1), (0, 1))), shape=(2, 2))
pauli_x = coo_array(([1, 1], ((0, 1), (1, 0))), shape=(2, 2))
pauli_y = coo_array(([-1j, 1j], ((0, 1), (1, 0))), shape=(2, 2))
pauli_z = coo_array(([1, -1], ((0, 1), (0, 1))), shape=(2, 2))

n_qubits = 10
n_dims = 2 ** n_qubits

In [3]:
all_x = coo_array(([0], ((0, ), (0, ))), shape=(n_dims, n_dims))
for i in range(n_qubits):
  x_i = pauli_x.copy()
  for _ in range(0, i):
    x_i = kron(pauli_0, x_i)
  for _ in range(i+1, n_qubits):
    x_i = kron(x_i, pauli_0)
  all_x += x_i

all_z = coo_array(([0], ((0, ), (0, ))), shape=(n_dims, n_dims))
for i, h_i in h.items():
  z_i = pauli_z.copy()
  for _ in range(0, i):
    z_i = kron(pauli_0, z_i)
  for _ in range(i+1, n_qubits):
    z_i = kron(z_i, pauli_0)
  all_z += h_i * z_i

hamiltonian = 0.5 * all_x + 0.5 * all_z

In [4]:
ones = np.ones((n_dims))

Q, T = lanczos_iteration(hamiltonian, ones, 5)

Print iteration 1.
Print iteration 2.
Print iteration 3.
Print iteration 4.
Print iteration 5.


In [5]:
Tk = Q.T.dot(hamiltonian).dot(Q)
print('Tk:', Tk)

Tk: [[<1024x1024 sparse array of type '<class 'numpy.float64'>'
  	with 11264 stored elements in Compressed Sparse Row format>
  <1024x1024 sparse array of type '<class 'numpy.float64'>'
  	with 11264 stored elements in Compressed Sparse Row format>
  <1024x1024 sparse array of type '<class 'numpy.float64'>'
  	with 11264 stored elements in Compressed Sparse Row format>
  <1024x1024 sparse array of type '<class 'numpy.float64'>'
  	with 11264 stored elements in Compressed Sparse Row format>
  <1024x1024 sparse array of type '<class 'numpy.float64'>'
  	with 11264 stored elements in Compressed Sparse Row format>
  <1024x1024 sparse array of type '<class 'numpy.float64'>'
  	with 11264 stored elements in Compressed Sparse Row format>]
 [<1024x1024 sparse array of type '<class 'numpy.float64'>'
  	with 11264 stored elements in Compressed Sparse Row format>
  <1024x1024 sparse array of type '<class 'numpy.float64'>'
  	with 11264 stored elements in Compressed Sparse Row format>
  <1024x102

In [7]:
print(Tk.shape)
print(Q.shape)
print(hamiltonian.shape)
print(type(Tk))
print(type(hamiltonian))
print(Q.T.dot(hamiltonian[:6, :6]))

(6, 6)
(1024, 6)
(1024, 1024)
<class 'numpy.ndarray'>
<class 'scipy.sparse._arrays.csr_array'>
[[<6x6 sparse array of type '<class 'numpy.float64'>'
  	with 20 stored elements in Compressed Sparse Row format>
  <6x6 sparse array of type '<class 'numpy.float64'>'
  	with 20 stored elements in Compressed Sparse Row format>
  <6x6 sparse array of type '<class 'numpy.float64'>'
  	with 20 stored elements in Compressed Sparse Row format> ...
  <6x6 sparse array of type '<class 'numpy.float64'>'
  	with 20 stored elements in Compressed Sparse Row format>
  <6x6 sparse array of type '<class 'numpy.float64'>'
  	with 20 stored elements in Compressed Sparse Row format>
  <6x6 sparse array of type '<class 'numpy.float64'>'
  	with 20 stored elements in Compressed Sparse Row format>]
 [<6x6 sparse array of type '<class 'numpy.float64'>'
  	with 20 stored elements in Compressed Sparse Row format>
  <6x6 sparse array of type '<class 'numpy.float64'>'
  	with 20 stored elements in Compressed Sparse 

In [15]:
ut, vt = np.linalg.eigh(Tk)
ut = np.sort(ut)
print('approximation:', ut)

UFuncTypeError: Cannot cast ufunc 'eigh_lo' input from dtype('O') to dtype('float64') with casting rule 'same_kind'

In [ ]:
u, v = np.linalg.eig(hamiltonian)
u = np.sort(u)
print('exact:', u)

In [ ]:
diffs = u[:len(ut)] - ut
print('diffs:', diffs.T)